In [10]:
import pandas as pd
import os

In [11]:
data = pd.read_csv('../DATA/wather_centr_2019-04-01_2019-12-08.csv',delimiter=';',
                   skiprows=6, parse_dates=['Местное время в Москве (центр, Балчуг)'], index_col=False)
data = data.rename(columns={'Местное время в Москве (центр, Балчуг)': 'date'})

In [12]:
data = data.set_index('date')
data = data[::-1]

In [13]:
sel_data = pd.DataFrame(index=data.index)

In [14]:
sel_data['temp_meteo'] = data['T']
sel_data['pres_meteo'] = data.Po *133.322 # transfor in Pa
sel_data['hum_meteo'] = data.U
sel_data['wind_direction'] = data.DD
sel_data['wind_speed'] = data.Ff
sel_data['precipitation'] = data.W1
sel_data['prec_amount'] = data.RRR
sel_data['visibility'] = data.VV
sel_data['dew_point_temp'] = data.Td

In [15]:
meteo_data = sel_data.resample('5T').pad()

In [16]:
DATA_FOLDER = '../DATA'
files = os.listdir(DATA_FOLDER)
sds_files = [i for i in files if 'sds011' in i]
bme_files = [i for i in files if 'bme280' in i]
dht_files = [i for i in files if 'dht22' in i]

In [17]:
with open('../processing_lists/sensorID_list.txt', 'r') as sensors:
    sensor_list = sensors.read().splitlines()
    indoor_sensors = [i for i in sensor_list if 'indoor' in i]

In [18]:
sds_files = [i for i in sds_files if i.split('_')[0] not in indoor_sensors]
bme_files = [i for i in bme_files if i.split('_')[0] not in indoor_sensors]
dht_files = [i for i in dht_files if i.split('_')[0] not in indoor_sensors]

In [19]:
data = pd.read_csv(os.path.join(
    DATA_FOLDER, '26717_bme280_sensor_.csv'), delimiter=';', parse_dates=['timestamp'], index_col=5)
data.head()

,sensor_id,sensor_type,location,lat,lon,pressure,altitude,pressure_sealevel,temperature,humidity
timestamp,,,,,,,,,,
2019-05-27 18:12:41,26717,BME280,14325,55.684,37.584,99031.29,NaN,NaN,18.46,43.94
2019-05-27 18:15:22,26717,BME280,14325,55.684,37.584,99037.25,NaN,NaN,18.39,44.21
2019-05-27 18:17:56,26717,BME280,14325,55.684,37.584,99032.41,NaN,NaN,18.33,44.31
2019-05-27 18:20:30,26717,BME280,14325,55.684,37.584,99032.87,NaN,NaN,18.35,44.31
2019-05-27 18:23:03,26717,BME280,14325,55.684,37.584,99031.01,NaN,NaN,18.33,44.49


In [20]:
sensors_data =[]

In [21]:
idx = pd.date_range('2019-04-01', '2019-12-08', freq='5T')

In [22]:
sds_data = pd.DataFrame(idx, columns=['date'])
sds_data = sds_data.set_index('date')

In [23]:
bme_data = pd.DataFrame(idx, columns=['date'])
bme_data = bme_data.set_index('date')

In [24]:
from pandas.errors import EmptyDataError
for f in bme_files:
    try:
        data = pd.read_csv(os.path.join(
            DATA_FOLDER, f), delimiter=';', parse_dates=['timestamp'], index_col=5)
        s_id = data.iloc[0].sensor_id
        s_type = data.iloc[0].sensor_type
        lat = data.iloc[0].lat
        lon = data.iloc[0].lon
        sensors_data.append([s_id, s_type, lat, lon])
        data['pressure'] = data.pressure.replace('unavailable', None)
        data['temperature'] = data.temperature.replace('unavailable', None)
        data['humidity'] = data.humidity.replace('unavailable', None)
        data['pressure'] = data.pressure.astype(float)
        data['temperature'] = data.temperature.astype(float)
        data['humidity'] = data.humidity.astype(float)
        new_data = data['2019-04':].resample('5T').mean()
        new_data.reindex(idx, fill_value=None)
        bme_data['pressure_'+str(s_id)] = new_data.pressure
        bme_data['temperature_'+str(s_id)] = new_data.temperature
        bme_data['humidity_'+str(s_id)] = new_data.humidity
    except EmptyDataError:
        pass

In [25]:
from pandas.errors import EmptyDataError
for f in sds_files:
    try:
        data = pd.read_csv(os.path.join(
            DATA_FOLDER, f), delimiter=';', parse_dates=['timestamp'], index_col=5)
        s_id = data.iloc[0].sensor_id
        s_type = data.iloc[0].sensor_type
        lat = data.iloc[0].lat
        lon = data.iloc[0].lon
        sensors_data.append([s_id, s_type, lat, lon])
        data['P1'] = data.P1.replace('unavailable', None)
        data['P2'] = data.P2.replace('unavailable', None)
        data['P1'] = data.P1.astype(float)
        data['P2'] = data.P2.astype(float)
        new_data = data['2019-04':].resample('5T').mean()
        new_data.reindex(idx, fill_value=None)
        sds_data['P1_'+str(s_id)] = new_data.P1
        sds_data['P2_'+str(s_id)] = new_data.P2
    except EmptyDataError:
        pass

/home/anatoly/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (6,9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [26]:
sensors = pd.DataFrame(sensors_data, columns=['sensor_id', 'sensor_type', 'lat', 'lon'])

In [28]:
p1_col = [i for i in sds_data.columns if 'P1_' in i]
p2_col = [i for i in sds_data.columns if 'P2_' in i]

In [29]:
temp_col = [i for i in bme_data.columns if 'temperature_' in i]
pres_col = [i for i in bme_data.columns if 'pressure_' in i]
hum_col = [i for i in bme_data.columns if 'humidity_' in i]

In [30]:
avg_data = pd.DataFrame(idx, columns=['date'])
avg_data = avg_data.set_index('date')

In [31]:
avg_data['P1'] = sds_data[p1_col].mean(axis = 1, skipna = True)
avg_data['P2'] = sds_data[p2_col].mean(axis = 1, skipna = True)
avg_data['pressure'] = bme_data[pres_col].mean(axis = 1, skipna = True)
avg_data['temperature'] = bme_data[temp_col].mean(axis = 1, skipna = True)
avg_data['humidity'] = bme_data[hum_col].mean(axis = 1, skipna = True)
avg_data['P1_std'] = sds_data[p1_col].std(axis = 1, skipna = True)
avg_data['P2_std'] = sds_data[p2_col].std(axis = 1, skipna = True)
avg_data['pressure_std'] = bme_data[pres_col].std(axis = 1, skipna = True)
avg_data['temperature_std'] = bme_data[temp_col].std(axis = 1, skipna = True)
avg_data['humidity_std'] = bme_data[hum_col].std(axis = 1, skipna = True)

In [35]:
for c in meteo_data.columns:
    avg_data[c] = meteo_data[c]

In [36]:
avg_data

,P1,P2,pressure,temperature,humidity,P1_std,P2_std,pressure_std,temperature_std,humidity_std,temp_meteo,pres_meteo,hum_meteo,wind_direction,wind_speed,precipitation,prec_amount,visibility,dew_point_temp
date,,,,,,,,,,,,,,,,,,,
2019-04-01 00:00:00,5.645000,3.137143,98513.240000,7.092500,55.912500,1.008224,0.709371,612.681767,1.196735,2.919511,-1.6,99644.8628,65,"Ветер, дующий с северо-запада",2,NaN,0.1,10.0,-7.4
2019-04-01 00:05:00,5.513333,3.040595,98521.940000,7.020000,56.035000,1.359509,0.797857,614.509696,1.230007,3.065667,-1.6,99644.8628,65,"Ветер, дующий с северо-запада",2,NaN,0.1,10.0,-7.4
2019-04-01 00:10:00,5.504643,3.011786,98522.760000,7.041250,55.885000,1.512618,0.933899,613.946187,1.265414,3.335688,-1.6,99644.8628,65,"Ветер, дующий с северо-запада",2,NaN,0.1,10.0,-7.4
2019-04-01 00:15:00,6.790714,3.461071,98529.485000,6.981250,55.907500,3.209377,1.644803,609.764213,1.211214,3.232178,-1.6,99644.8628,65,"Ветер, дующий с северо-запада",2,NaN,0.1,10.0,-7.4
2019-04-01 00:20:00,7.344643,4.065357,98529.307500,6.978750,55.855000,7.148042,3.540958,615.146656,1.342723,3.579527,-1.6,99644.8628,65,"Ветер, дующий с северо-запада",2,NaN,0.1,10.0,-7.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-07 23:40:00,31.604496,16.724785,96175.438856,0.885327,62.658170,9.130953,4.305813,13079.228381,3.891334,21.121382,14.3,99364.8866,75,"Ветер, дующий с севера",1,NaN,0.3,10.0,9.9
2019-12-07 23:45:00,31.576674,16.789371,96173.823152,0.920501,62.612167,9.205834,4.293148,13078.971667,3.890731,21.112555,14.3,99364.8866,75,"Ветер, дующий с севера",1,NaN,0.3,10.0,9.9
2019-12-07 23:50:00,31.272443,16.667139,96215.129219,1.127130,62.229471,8.544895,4.141062,12963.114051,4.066458,21.056658,14.3,99364.8866,75,"Ветер, дующий с севера",1,NaN,0.3,10.0,9.9


In [38]:
avg_data.to_csv('dataset.csv')

In [39]:
avg_data

,P1,P2,pressure,temperature,humidity,P1_std,P2_std,pressure_std,temperature_std,humidity_std,temp_meteo,pres_meteo,hum_meteo,wind_direction,wind_speed,precipitation,prec_amount,visibility,dew_point_temp
date,,,,,,,,,,,,,,,,,,,
2019-04-01 00:00:00,5.645000,3.137143,98513.240000,7.092500,55.912500,1.008224,0.709371,612.681767,1.196735,2.919511,-1.6,99644.8628,65,"Ветер, дующий с северо-запада",2,NaN,0.1,10.0,-7.4
2019-04-01 00:05:00,5.513333,3.040595,98521.940000,7.020000,56.035000,1.359509,0.797857,614.509696,1.230007,3.065667,-1.6,99644.8628,65,"Ветер, дующий с северо-запада",2,NaN,0.1,10.0,-7.4
2019-04-01 00:10:00,5.504643,3.011786,98522.760000,7.041250,55.885000,1.512618,0.933899,613.946187,1.265414,3.335688,-1.6,99644.8628,65,"Ветер, дующий с северо-запада",2,NaN,0.1,10.0,-7.4
2019-04-01 00:15:00,6.790714,3.461071,98529.485000,6.981250,55.907500,3.209377,1.644803,609.764213,1.211214,3.232178,-1.6,99644.8628,65,"Ветер, дующий с северо-запада",2,NaN,0.1,10.0,-7.4
2019-04-01 00:20:00,7.344643,4.065357,98529.307500,6.978750,55.855000,7.148042,3.540958,615.146656,1.342723,3.579527,-1.6,99644.8628,65,"Ветер, дующий с северо-запада",2,NaN,0.1,10.0,-7.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-07 23:40:00,31.604496,16.724785,96175.438856,0.885327,62.658170,9.130953,4.305813,13079.228381,3.891334,21.121382,14.3,99364.8866,75,"Ветер, дующий с севера",1,NaN,0.3,10.0,9.9
2019-12-07 23:45:00,31.576674,16.789371,96173.823152,0.920501,62.612167,9.205834,4.293148,13078.971667,3.890731,21.112555,14.3,99364.8866,75,"Ветер, дующий с севера",1,NaN,0.3,10.0,9.9
2019-12-07 23:50:00,31.272443,16.667139,96215.129219,1.127130,62.229471,8.544895,4.141062,12963.114051,4.066458,21.056658,14.3,99364.8866,75,"Ветер, дующий с севера",1,NaN,0.3,10.0,9.9
